In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
# downsample_results = pd.read_csv("./SNU_downsample_results/results_table01.tsv", sep = " ")
# original_results = pd.read_csv("./SNU_downsample_results/results_table_original.tsv", sep = " ")

# helper functions

In [2]:
def create_bins(results_df):
    start_list = [str(d) for d in list(results_df["start"])]
    end_list = [str(d) for d in list(results_df["end"])]
    results_df["bin"] = results_df["seq"] + start_list + end_list
    return results_df

In [3]:
def create_df_subsets(downsample_path, original_path):
    downsample_results = pd.read_csv(downsample_path, sep = " ")
    original_results = pd.read_csv(original_path, sep = " ")

    # add bin labels
    downsample_results = create_bins(downsample_results)
    original_results = create_bins(original_results)

    # get subset of bins + cells that should be kept
    bin_subset = set(list(downsample_results["bin"])).intersection(set(list(original_results["bin"])))
    cell_subset = set(list((downsample_results.columns))).intersection(set(list(original_results.columns)))

    # subset both
    downsample_subset = downsample_results[downsample_results.columns[downsample_results.columns.isin(cell_subset)]].loc[downsample_results["bin"].isin(bin_subset)].reset_index(drop=True)
    original_subset = original_results[original_results.columns[original_results.columns.isin(cell_subset)]].loc[original_results["bin"].isin(bin_subset)].reset_index(drop=True)

    # make sure column names match
    downsample_subset = downsample_subset[original_subset.columns.tolist()]
    
    return[downsample_subset, original_subset, original_results]
    

In [4]:
def get_statistics(ds_subset, og_subset, original_results):
    
    ds_call_array = np.array(ds_subset.iloc[:,3:-1])
    og_call_array = np.array(og_subset.iloc[:,3:-1])
    original_results_call_array = np.array(original_results.iloc[:,3:-1])
    
    # all calls labeled normal
    original_normal = len(np.where((original_results_call_array == 1)[0]))
    TP_normal = len(np.where((og_call_array == 1) & (ds_call_array == 1))[0]) # both called
    FP_normal = len(np.where((og_call_array != 1) & (ds_call_array == 1))[0]) # subset called normal; original did not
    FN_normal = len(np.where((og_call_array == 1) & (ds_call_array != 1))[0]) # subset did not call normal; original did
    TN_normal = len(np.where((og_call_array != 1) & (ds_call_array != 1))[0]) # both said was not normal

#     print(TP_normal)
#     print(FP_normal)
#     print(FN_normal)
#     print(TN_normal)
    
    percent_normal = TP_normal/original_normal
    precision_normal = TP_normal/(TP_normal+FP_normal)
    recall_normal = TP_normal/(TP_normal+FN_normal)
    F1_normal = 2*((precision_normal*recall_normal)/(precision_normal+recall_normal))
    specificity_normal = TN_normal/(TN_normal+FP_normal)
    
    # all calls labeled loss
    original_loss = len(np.where((original_results_call_array == 0)[0]))
    TP_loss = len(np.where((og_call_array == 0) & (ds_call_array == 0))[0]) # both called
    FP_loss = len(np.where((og_call_array != 0) & (ds_call_array == 0))[0]) # subset called loss; original did not
    FN_loss = len(np.where((og_call_array == 0) & (ds_call_array != 0))[0]) # subset did not call loss; original did
    TN_loss = len(np.where((og_call_array != 0) & (ds_call_array != 0))[0]) # both said was not loss

    percent_loss = TP_loss/original_loss
    precision_loss = TP_loss/(TP_loss+FP_loss)
    recall_loss = TP_loss/(TP_loss+FN_loss)
    F1_loss = 2*((precision_loss*recall_loss)/(precision_loss+recall_loss))
    specificity_loss = TN_loss/(TN_loss+FP_loss)
    
    # all calls labeled gain 
    original_gain = len(np.where((original_results_call_array > 1)[0]))
    TP_gain = len(np.where((og_call_array > 1) & (og_call_array == ds_call_array))[0]) # both called
    FP_gain = len(np.where((og_call_array <= 1) & (ds_call_array > 1))[0]) # subset called gain; original did not
    FN_gain = len(np.where((og_call_array > 1) & (ds_call_array <= 1))[0]) # subset did not call gain; original did
    TN_gain = len(np.where((og_call_array <= 1) & (ds_call_array <= 1))[0]) # both said was not gain

    percent_gain = TP_gain/original_gain
    precision_gain = TP_gain/(TP_gain+FP_gain)
    recall_gain = TP_gain/(TP_gain+FN_gain)
    F1_gain = 2*((precision_gain*recall_gain)/(precision_gain+recall_gain))
    specificity_gain = TN_gain/(TN_gain+FP_gain)
    
    statistic_dictionary = {}
    statistic_dictionary["loss"] = {"percent_call": percent_loss, "precision": precision_loss, "recall": recall_loss, "specificity": specificity_loss, "F1": F1_loss}
    statistic_dictionary["normal"] = {"percent_call": percent_normal, "precision": precision_normal, "recall": recall_normal, "specificity": specificity_normal,"F1": F1_normal}
    statistic_dictionary["gain"] = {"percent_call": percent_gain, "precision": precision_gain, "recall": recall_gain, "specificity": specificity_gain, "F1": F1_gain}

    return statistic_dictionary

# get stats

In [5]:
ds_path = "./SNU_downsample_results/results_table01.tsv"
og_path = "./SNU_downsample_results/results_table_original.tsv"

# subset_dfs = create_df_subsets(ds_path, og_path)
# ds_subset = subset_dfs[0]
# og_subset = subset_dfs[1]

In [8]:
statistic_dict = {}
statistic_df = pd.DataFrame(columns=["downsample_frac", "mean_depth", "event", "percent_call", "precision", "recall", "specificity", "F1"])

for i in tqdm(range(1,10)):
    
#for i in range(1,2):
    
    print(i)
    
    # get read depth
    ds_fragment_counts_path = f"./epiAneufinder/SNU_downsample_files/fragment_files/fragments_filtered_00{i}.tsv"
    fragments_counts = pd.read_csv(ds_fragment_counts_path, sep = "\t", header=None)
    
    mean_depth = (fragments_counts).groupby([3]).size().mean()
    
    # get read statistics
    ds_path = f"./SNU_downsample_files/2k_results/results_table_00{i}.tsv"
    og_path = "./SNU_downsample_files/2k_results/results_table_original.tsv"
    
    subset_dfs = create_df_subsets(ds_path, og_path)
    ds_subset = subset_dfs[0]
    og_subset = subset_dfs[1]
    original_results = subset_dfs[2]
    
    print(og_subset.shape)
    print(original_results.shape)
    
    statistic_dict[f"0.0{i}"] = get_statistics(ds_subset, og_subset, original_results)
    
    statistic_df.loc[len(statistic_df.index)] = [f"0.0{i}", 
                                                 mean_depth,
                                                 "loss", 
                                                 statistic_dict[f"0.0{i}"]["loss"]["percent_call"],
                                                 statistic_dict[f"0.0{i}"]["loss"]["precision"], 
                                                 statistic_dict[f"0.0{i}"]["loss"]["recall"], 
                                                 statistic_dict[f"0.0{i}"]["loss"]["specificity"], 
                                                 statistic_dict[f"0.0{i}"]["loss"]["F1"]]
                                                 
    statistic_df.loc[len(statistic_df.index)] = [f"0.0{i}", 
                                                 mean_depth,
                                                 "normal", 
                                                 statistic_dict[f"0.0{i}"]["normal"]["percent_call"],
                                                 statistic_dict[f"0.0{i}"]["normal"]["precision"], 
                                                 statistic_dict[f"0.0{i}"]["normal"]["recall"], 
                                                 statistic_dict[f"0.0{i}"]["normal"]["specificity"], 
                                                 statistic_dict[f"0.0{i}"]["normal"]["F1"]]
                                                 
    statistic_df.loc[len(statistic_df.index)] = [f"0.0{i}", 
                                                 mean_depth,
                                                 "gain", 
                                                 statistic_dict[f"0.0{i}"]["gain"]["percent_call"],
                                                 statistic_dict[f"0.0{i}"]["gain"]["precision"], 
                                                 statistic_dict[f"0.0{i}"]["gain"]["recall"], 
                                                 statistic_dict[f"0.0{i}"]["gain"]["specificity"], 
                                                 statistic_dict[f"0.0{i}"]["gain"]["F1"]]


  0%|                                                     | 0/9 [00:00<?, ?it/s]

1
(2618, 118)
(25976, 2467)



 11%|█████                                        | 1/9 [00:09<01:17,  9.68s/it]

2
(5554, 428)
(25976, 2467)



 22%|██████████                                   | 2/9 [00:15<00:49,  7.12s/it]

...


 78%|███████████████████████████████████          | 7/9 [01:42<00:37, 18.53s/it]

8
(14320, 1715)
(25976, 2467)



 89%|████████████████████████████████████████     | 8/9 [02:04<00:19, 19.93s/it]

9
(15640, 1397)
(25976, 2467)


100%|█████████████████████████████████████████████| 9/9 [02:30<00:00, 16.71s/it]


In [9]:
statistic_df.to_csv("downsample_statistics_00x_REDO.csv")